In [3]:
%load_ext cython
%matplotlib inline
import sys
sys.path.append('/home/bram/ANTS')
sys.path.append('/home/bram/ANTS/entropy')
for p in sys.path:
    print(p)


/usr/lib/python36.zip
/usr/lib/python3.6
/usr/lib/python3.6/lib-dynload
/home/bram/PythonEnvs/Ants/lib/python3.6/site-packages
/home/bram/PythonEnvs/Ants/lib/python3.6/site-packages/IPython/extensions
/home/bram/.ipython
/home/bram/ANTS
/home/bram/ANTS/entropy


In [49]:
%%cython -a

from cythonic.plugins.positions cimport point
from cythonic.plugins.functions cimport transform
from libc.math cimport M_PI as PI
cimport cython

cdef struct observations:
    double lft
    double rght
    
cdef struct AntState:
    # " position and orientation "
    unsigned int id
    point pos #{x,y} in mm
    point left #left sensor location
    point right #right sensor location
    double theta #azimuth in degrees
    double omega #angular rotation in degrees/second
    double v # mm/s
    observations Q_obs
    
    #" ant status "
    bint foodbound
    bint active
    bint out_of_bounds
    
    #" individual timers "
    double rng_timer # for the random number generator
    double time #time since last event
    
cdef AntState X
X.id = 1

cdef struct fun_args:
    double gain
    double breakpoint
    
# function type for observe function
ctypedef void (*f_obs)(AntState*,fun_args*, observations*)

@cython.cdivision(True)
@cython.wraparound(False)
@cython.boundscheck(False)
@cython.nonecheck(False)
@cython.initializedcheck(False)
cdef void observe_linear(AntState* s, fun_args* a, observations* Q):
    s[0].Q_obs.lft = Q[0].lft*a[0].gain
    s[0].Q_obs.rght = Q[0].rght*a[0].gain

@cython.cdivision(True)
@cython.wraparound(False)
@cython.boundscheck(False)
@cython.nonecheck(False)
@cython.initializedcheck(False)
cdef class Ant:
    cdef AntState* state
    cdef:
        # " geometric properties "
        double l
        double sens_offset
        
        # sensor properties
        double gain
        f_obs sens_fun
        fun_args obs_fun_args
        
        
        # actuator properties
        double drop_quantity
        double return_factor
        double drop_beta
    
    def __cinit__(self, double l, double sens_offset, double gain, double drop_quantity, double return_factor,
                 double drop_beta):
        self.l = l
        self.sens_offset = sens_offset
        self.gain = gain
        self.drop_quantity = drop_quantity
        self.return_factor = return_factor
        self.drop_beta = drop_beta
        self.sens_fun = observe_linear
        self.obs_fun_args.gain = 1
        
    cdef void rotate(self,double* dt):
        " compute the angular speed (degrees/second) based on sensing "
        self.state[0].omega = self.gain*180./PI*(
            self.state[0].Q_obs.lft-self.state[0].Q_obs.rght)
        self.increase_azimuth(dt)
        
    cdef void gradient_step(self, double *dt, observations Q):
        " execute the sequence for differential based stepping "
        " sniff pheromone -> rotate -> step -> update sensors "
        
        self.observe(&Q)
        self.rotate(dt)
        self.step(dt)
        self.set_sensors()
        
    cdef void observe(self, observations* Q):
        # fill Q_obs(lft, right)
        self.sens_fun(self.state, &self.obs_fun_args, Q)
        
    cdef void step(self, double * dt):
        " do a step in the current direction ,do boundary check as well "
        self.state[0].time += dt[0] # update internal timer
        
        # update position
        cdef double step_size = self.state[0].v*dt[0]
        self.state[0].pos = transform(self.state[0].theta, &step_size, &self.state.pos)
        
    cdef void out_of_bounds(self, bint oob):
        " toggle out of bounds status "
        self.state[0].out_of_bounds = oob
        
    cdef void set_sensors(self):
        " calculate the position of the left and right sensor antennas "
        self.state[0].left = transform(self.state[0].theta + self.sens_offset, &self.l, &self.state.pos)
        self.state[0].right = transform(self.state[0].theta - self.sens_offset, &self.l, &self.state.pos)
        
    cdef void activate(self):
        " mark the ant as active "
        self.state[0].active = True
    
    cdef void increase_azimuth(self, double * dt):
        " ensure azimuth stays within [0,360) interval "
        " increase azimuth based on the angular speed and the time interval "
        self.state[0].theta = (self.state[0].theta+self.state[0].omega*dt[0])%360

        
    cdef void set_state(self,AntState* s):
        self.state = s

cdef Ant ant = Ant(l  =10., sens_offset = 45, gain = 1., drop_quantity = 1., return_factor = 1.5, drop_beta = 2.,
           )

cdef double x = 0.
cdef double dt = .1
cdef observations Q
Q.lft = 1
Q.rght = 2
X.v = 10
ant.set_state(&X)
ant.activate()
ant.set_sensors()
ant.gradient_step(&dt, Q = Q)
print(X)

{'id': 1, 'pos': {'x': 0.9950041652780257, 'y': -0.09983341664682817}, 'left': {'x': 8.73667495004749, 'y': 6.229979650122753}, 'right': {'x': 7.324817232047607, 'y': -7.841504201416293}, 'theta': -5.729577951308233, 'omega': -57.29577951308232, 'v': 10.0, 'Q_obs': {'lft': 1.0, 'rght': 2.0}, 'foodbound': False, 'active': True, 'out_of_bounds': False, 'rng_timer': 0.0, 'time': 0.1}


In [50]:
%%cython -a

cdef char* s = b'string'
if s == b"string":
    print(s)

b'string'
